# Solution strategies
This tutorials demonstrates some of the more advanced solution strategies available in `PorePy`. 
The model problems are presented succinctly.

## Line search for multiphysics problems with fracture deformation
The combination of highly nonlinear multiphysics problems and the non-smooth formulation of contact mechanics may lead to severe convergence issues.
To improve convergence, we provide a line search algorithm tailored to the irregularities of the contact equations, see https://arxiv.org/abs/2407.01184.
We demonstrate how to invoke the functionality to achieve convergence in a simple case which defies convergence for a regular Newton algorithm.

To define a suitably challenging problem containing open, sticking and slipping fracture cells, we prescribe heterogeneous Dirichlet values for displacement on a unit cube domain containing a single fracture. 
We also material parameters for granite and water, with bespoke (nonzero) values for fracture deformation parameters.

In [8]:
import porepy as pp
import numpy as np

from porepy.applications.md_grids.model_geometries import CubeDomainOrthogonalFractures
from porepy.applications.boundary_conditions.model_boundary_conditions import BoundaryConditionsMechanicsDirNorthSouth
from porepy.numerics.nonlinear import line_search

class DisplacementBoundaryConditions(BoundaryConditionsMechanicsDirNorthSouth):
    time_manager: pp.TimeManager

    def bc_values_displacement(self, boundary_grid: pp.BoundaryGrid) -> np.ndarray:
        """Boundary values for mechanics.

        Parameters:
            subdomains: List of subdomains on which to define boundary conditions.

        Returns:
            Array of boundary values.

        """

        # Default is zero.
        vals = np.zeros((self.nd, boundary_grid.num_cells))
        if boundary_grid.dim < self.nd - 1:
            return vals.ravel("F")
        boundary_faces = self.domain_boundary_sides(boundary_grid)

        faces = boundary_faces.north
        # Start by offsetting by the fracture gap.
        vals[1, faces] = self.solid.fracture_gap()
        if self.time_manager.time < 1e-5:
            return vals.ravel("F")

        u_char = self.characteristic_displacement([boundary_grid]).value(
            self.equation_system
        )
        # Normal component of boundary displacement.
        # Produce different contact regimes along the fracture.
        coo = (
            boundary_grid.cell_centers[0, faces]
            + boundary_grid.cell_centers[2, faces]
        ) / np.max(self.domain.side_lengths())
        linear_increase = u_char * (coo - 0.5)
        offset = -0.5 * u_char
        vals[1, faces] += offset - 2.0 * linear_increase
        # Tangential component of boundary displacement.
        w = self.params.get("tangential_u_weight", 2)
        vals[0, faces] = w * u_char * 2.0
        vals[1, faces] = w * u_char * 1.0
        return vals.ravel("F")

class NonzeroInitialCondition:
    """A decent initial condition for the contact problem greatly improves convergence."""
    def initial_condition(self) -> None:
        """Set the initial condition for the problem."""
        super().initial_condition()
        for var in self.equation_system.variables:
            if hasattr(self, "initial_" + var.name):
                values = getattr(self, "initial_" + var.name)([var.domain])
                self.equation_system.set_variable_values(
                    values, [var], iterate_index=0, time_step_index=0
                )

    def initial_t(self, subdomains: list[pp.Grid]) -> pp.ad.Operator:
        """Initial contact traction [Pa].

        Parameters:
            subdomains: List of subdomains.

        Returns:
            Operator for initial contact traction.

        """
        sd = subdomains[0]
        traction_vals = np.zeros((self.nd, sd.num_cells))
        traction_vals[-1] = -self.characteristic_contact_traction(subdomains).value(
            self.equation_system
        ) / self.params.get("characteristic_displacement_scaling", 1.0)
        return traction_vals.ravel("F")




class TailoredPoromechanics(
    DisplacementBoundaryConditions,
    CubeDomainOrthogonalFractures,
    pp.constitutive_laws.CubicLawPermeability,
    pp.models.solution_strategy.ContactIndicators,
    NonzeroInitialCondition,
    pp.poromechanics.Poromechanics,
):
    """Combine mixins with the poromechanics class."""

class NonlinearSolver(
    line_search.ConstraintLineSearch,
    line_search.SplineInterpolationLineSearch,
    line_search.LineSearchNewtonSolver,
):
    """Collect all the line search methods in one class."""

granite_values: dict[str, float] = pp.solid_values.granite
granite_values.update(
    {
        "permeability": 2e-15,
        "normal_permeability": 1e-7,
        "residual_aperture": 2e-5,
        "dilation_angle": 0.2,
        "characteristic_displacement": 1e-2,
    }
)
params = {
    "times_to_export": [], # No output
    "material_constants": {"solid":  pp.SolidConstants(granite_values), "fluid": pp.FluidConstants(pp.fluid_values.water)},
    "fracture_indices": [1], # Fracture with constant y-coordinate
    "meshing_arguments": {"cell_size": 1 / 4},
    # Specify iterative solver parameters
    "max_iterations": 30,
    "nl_convergence_tol": 1e-7,  # Limited by condition number
    "nl_convergence_tol_res": 1e-7,
    "linear_solver": "scipy_sparse",
}


To invoke the line search, we need to specify a few parameters, including the "nonlinear_solver". 
We do this in a copy of the parameter dictionary, allowing us to run two simulations.

In [9]:
params.update(
    {
        "nonlinear_solver": NonlinearSolver,
        "Local_line_search": 1,  # Set to 0 to use turn off the tailored line search
        "Global_line_search": 0,  # Set to 1 to use turn on a residual-based line search
        "adaptive_indicator_scaling": 1,
        "time_manager": pp.TimeManager(
            [0.0, 1e-3],  # Time interval
            1e-3,  # Time step
            True,
        ),
        }
)

model = TailoredPoromechanics(params)
pp.run_time_dependent_model(model, params)
print("Converged in {} iterations".format(model.nonlinear_solver_statistics.num_iteration))

Converged in 18 iterations


Some known issues to consider:
- The nonlinearity caused by the fracture permeability has proven sensitive to discretization method.
TPFA seems considerably more stable than MPFA.
For details, see the above mentioned paper.
- If regular Newton converges, it may be faster than the line search method.
An unexplored option is to use the line search as a fallback if Newton fails to converge.